# Approximate Nearest Neighbors:

# Image Recommendation System via Collaborative Filtering

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”.Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.

### Import necessary libraries

In [ ]:
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
import numpy as np
import pandas as pd
import faiss
from scipy import spatial
from sklearn.decomposition import TruncatedSVD

In [ ]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [ ]:
config = pd.read_csv('config.csv').iloc[0]

In [ ]:
config

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object

In [ ]:
user = config['id']
k_value = config['k']

### Read the Data

In [ ]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

In [ ]:
# your code here
# appreciate_data = appreciate_data[:1000]

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [ ]:
user_likes = dict()

In [ ]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    if user_id not in user_likes:
        user_likes[user_id] = list()
    
    user_likes[user_id].append(item_id)

In [ ]:
# user_likes

### Use your choice of Approximate Nearest Neigbor after Collaborative Filtering to find nearest neighbors

In [ ]:
unique_item_id = set()
for keys in user_likes:
  temp = user_likes[keys]
  for value in temp:
    unique_item_id.add(value)
unique_user_id = set()
for keys in user_likes:
  unique_user_id.add(keys)

In [ ]:
user_map = {}
item_map = {}
for i,user in enumerate(list(unique_user_id)):
  user_map[user] = i
for i,item in enumerate(list(unique_item_id)):
  item_map[item] = i
# print(user_map)
# print(item_map)

In [ ]:
matrix = [[0 for i in range(len(unique_item_id))] for j in range(len(unique_user_id))]
# print(len(matrix))
for i in user_likes: 
  temp = user_likes[i]
  for j in temp:
    matrix[user_map[i]][item_map[j]] += 1

In [ ]:
svd = TruncatedSVD(n_components=500)
new_matrix = svd.fit_transform(matrix)
# print(len(new_matrix))
# print(len(new_matrix[0]))

In [ ]:
new_matrix = np.asarray(new_matrix, np.float32)
# print(type(new_matrix))
# print(new_matrix.shape)
k = 5
d = 359
xq = new_matrix
index = faiss.index_factory(d, "Flat")
index.train(new_matrix)
index.add(new_matrix)
distances, neighbors = index.search(xq, k)
# print(distances)
# print(neighbors)

### Answer the following questions:

#### Q1. **Explain how your choice of library works**

The library I have used is `faiss`. <br />
Given a set of vectors x_i in some dimension d, Faiss creates a data structure in RAM. Once the structure is built, it efficiently performs the function of finding the minimum Euclidean distance between any two vectors given any new vector x in dimension d, that is,
it efficiently performs the following function:
i = argmin_i ||x-x_i|| <br/>
Computing the argmin is the search operation on the index where the index is the data structure created by Faiss.
The data structure in Faiss is an object which has an add method that adds x_i vectors (where x_i's are fixed). <br/>
faiss can search several vectors at the same time and stores the index on disk instead of in RAM. 



#### Q2. **Compare your choice of library with vanilla KNN.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

The time complexity for KNN is O(1) for training and O(n) for testing thus it depends on the number of test examples. <br />
The time complexity of KNN is `O(n*m)` for the brute-force neighbour search where n is the training examples and m is the number of dimensions in the training set whereas the underlying algorithmic novelty behind faiss takes an `O(nlog2n)` sort and parallelizes it to something that takes `O(log2n)` serial time. <br />
The problem in KNN arises when classes overlap and when sample size is unevenly distributed between categories thus requiring methods that increase classification rate in terms of recall measure.

#### Q3. **Compare your choice of library with implementation of ScaNN, faiss and annoy.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

faiss returns not just the nearest neighbour but also the k-th nearest neighbors. It can trade precision for speed. For instance, by giving an incorrect answer 10% of the time using a method that uses 10x less memory. 
The underlying algorithmic novelty behind faiss takes an O(nlog2n) sort and parallelizes it to something that takes O(log2n) serial time. <br />
On the other hand, Annoy requires two main parameters namely number of tress `n_trees` and the number of nodes to inspect during searching `search_k`. <br />
`n_trees` affects the build time and the index size whereas `search_k` affects the search performance. <br />
Annoy comes with huge speed but does not guarantee to find the nearest one, it approximates. It reduces the time complexity to `O(logn)`<br />
ScaNN includes search space pruning and quantization for Maximum Inner Product Search and also supports other distance functions. 

### Open the output file to write all the lines to the file

In [ ]:
outFile = open(config['output_file'], 'w')

inv_user = {v: k for k, v in user_map.items()}

for i,neighbor in enumerate(neighbors):
  for j,users in enumerate(neighbor):
    if j == 0:
      continue
    for item_id in user_likes[inv_user[users]]:
      if item_id not in user_likes[inv_user[i]]:
        outFile.write(str(item_id) + ' ' + str(inv_user[users]) + '\n')

outFile.close()